## Experiment 1: (2022-07-28~)
- - -
#### 0. Preprocessing / (Done on seperate file)
Urbansound8k Dataset i){8732(<=4s).wav files/10 folders(must be separated)} => Noise data    
MTG-jamendo Dataset i){}=> Music data   
For easy labeling, we need to save the labels along with the data ( X.npy = [ data , label ] )

#### 1. Initialize input pipeline


#### 2. Model initialization


#### 3. Training & Evaluation

- - -

In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, losses

import pathlib

import librosa

from IPython.display import clear_output

autotune = tf.data.AUTOTUNE

In [4]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            #tf.config.experimental.set_virtual_device_configuration(gpu, [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024*11)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


## 1. Initializing Input Pipeline   
(planned 22.08.08)   
* First define the raw tf.data.Datasets.   
* Make a function that preprocesses the training datasets.
* Use tf.data.Dataset.map to create the fully processed dataset.
* Then make a function that finds the labels for each of the training datasets.   
* Use tf.data.Dataset.map to create the label Dataset.   
* Then combine them with tf.data.Dataset.zip.   
* Then Shuffle & Configure for performance

In [5]:
IMAGE_SIZE = 513*345
IMAGE_DIM = [513,345,1]

# Training
batch_size = 1

In [6]:
base_path = pathlib.Path('D:/전재원졸업연구/music-discern/') # Edit on other computers
base_path_dataset = pathlib.Path('D:/전재원졸업연구/music-discern/Datasets/') # Edit on other computers

#Urbansound is nested in Datasets/Urbansound8K/audio/fold**
#Urbansound is already separated into batches, which need to be preserved.
urban_metadata_path = pathlib.Path(base_path_dataset/'Urbansound8K/metadata')
urban_audio_folders_path = pathlib.Path(base_path_dataset/'Urbansound8K/processed')
urban_audio_path_list = [x for x in urban_audio_folders_path.iterdir()]
temp = urban_audio_path_list.pop(1)
urban_audio_path_list.append(temp)
urban_audio_path_list = urban_audio_path_list[:4]
#Jamendo is nested in Datasets/MTGJamendo/audio
#Jamendo is not separated in batches, so we need to separate it into 10 batches, matching urbansound.
jamendo_metadata_path = pathlib.Path(base_path_dataset/'MTGJamendo/metadata')
jamendo_audio_folders_path = pathlib.Path(base_path_dataset/'MTGJamendo/processed')
jamendo_audio_path_list = [x for x in jamendo_audio_folders_path.iterdir()]
temp = jamendo_audio_path_list.pop(1)
jamendo_audio_path_list.append(temp)
jamendo_audio_path_list = jamendo_audio_path_list[:4]

#for later -> add a path for any .wav file that I want to observe.
test_subjects_path = pathlib.Path("./")

save_weights_path = pathlib.Path(base_path/'Experiment1/model_checkpoints')
save_weights_A_path = pathlib.Path(save_weights_path/'model_A')
save_weights_B_path = pathlib.Path(save_weights_path/'model_B')

Train = [0,1,2]
Test = [3] #Seperation, for later evaluation in Training step B

print(urban_audio_path_list)
print(jamendo_audio_path_list)

[WindowsPath('D:/전재원졸업연구/music-discern/Datasets/Urbansound8K/processed/fold1'), WindowsPath('D:/전재원졸업연구/music-discern/Datasets/Urbansound8K/processed/fold2'), WindowsPath('D:/전재원졸업연구/music-discern/Datasets/Urbansound8K/processed/fold3'), WindowsPath('D:/전재원졸업연구/music-discern/Datasets/Urbansound8K/processed/fold4')]
[WindowsPath('D:/전재원졸업연구/music-discern/Datasets/MTGJamendo/processed/fold1'), WindowsPath('D:/전재원졸업연구/music-discern/Datasets/MTGJamendo/processed/fold2'), WindowsPath('D:/전재원졸업연구/music-discern/Datasets/MTGJamendo/processed/fold3'), WindowsPath('D:/전재원졸업연구/music-discern/Datasets/MTGJamendo/processed/fold4')]


In [7]:
dataset_for_training_A = "empty"
dataset_for_training_B_train = "empty"
dataset_for_training_B_test = "empty"

dataset_for_training_A = tf.data.Dataset.list_files(str(urban_audio_folders_path/'*/*.npy'))
dataset_for_training_A = dataset_for_training_A.concatenate( tf.data.Dataset.list_files(str(jamendo_audio_folders_path/'*/*.npy')) )

for k in Train:
    temp_path_urban = str(urban_audio_path_list[k])
    temp_path_jam = str(jamendo_audio_path_list[k])
    if dataset_for_training_B_train == "empty":
        dataset_for_training_B_train = tf.data.Dataset.list_files(temp_path_urban + '/*.npy')
        dataset_for_training_B_train = dataset_for_training_B_train.concatenate( tf.data.Dataset.list_files(temp_path_jam + '/*.npy') )
    else:
        dataset_for_training_B_train = dataset_for_training_B_train.concatenate( tf.data.Dataset.list_files(temp_path_urban + '/*.npy') )
        dataset_for_training_B_train = dataset_for_training_B_train.concatenate( tf.data.Dataset.list_files(temp_path_jam + '/*.npy') )

for k in Test:
    temp_path_urban = str(urban_audio_path_list[k])
    temp_path_jam = str(jamendo_audio_path_list[k])
    if dataset_for_training_B_test == "empty":
        dataset_for_training_B_test = tf.data.Dataset.list_files(temp_path_urban + '/*.npy')
        dataset_for_training_B_test = dataset_for_training_B_test.concatenate( tf.data.Dataset.list_files(temp_path_jam + '/*.npy') )
    else:
        dataset_for_training_B_test = dataset_for_training_B_test.concatenate( tf.data.Dataset.list_files(temp_path_urban + '/*.npy') )
        dataset_for_training_B_test = dataset_for_training_B_test.concatenate( tf.data.Dataset.list_files(temp_path_jam + '/*.npy') )

# returns dataset for training A, dataset for training B Train, dataset for training B Test

In [8]:
for f in dataset_for_training_A.take(1):
    print(f)
    print(tf.strings.split( tf.strings.split( f,os.sep )[-1],sep="_" )[1])
    A = np.load(f.numpy(),allow_pickle=True)
    print(A.dtype)
    img = tf.io.read_file(f)
    img = tf.io.decode_raw(img,tf.complex64)
    img = tf.slice(img,begin=[16],size=[IMAGE_SIZE])
    img = tf.reshape(img,[513,345])
    print(img)

tf.Tensor(b'D:\\\xec\xa0\x84\xec\x9e\xac\xec\x9b\x90\xec\xa1\xb8\xec\x97\x85\xec\x97\xb0\xea\xb5\xac\\music-discern\\Datasets\\Urbansound8K\\processed\\fold10\\180127-4-0-1_noise_0.npy', shape=(), dtype=string)
tf.Tensor(b'noise', shape=(), dtype=string)
complex64
tf.Tensor(
[[ 3.50085385e-02+0.0000000e+00j  5.02819344e-02-2.1028942e-01j
   5.74264705e-01+4.0884653e-01j ...  1.63896400e-02-1.3452653e-02j
  -8.55796319e-03+2.1257186e-02j  1.00550218e-03-1.3767668e-02j]
 [-7.13702897e-03+7.6926346e-03j  1.15916887e-02-1.1925502e-02j
  -9.76558588e-03+1.5707988e-02j ...  3.32758754e-01+2.8869955e-02j
  -2.64837116e-01+1.2721957e-01j  1.07951805e-01-3.2055101e-01j]
 [ 3.34987836e-03+3.4452245e-01j -3.83727588e-02-2.3163638e-01j
   4.20296192e-02+7.8344174e-02j ...  1.53636014e+00+1.2098880e-01j
  -1.81544614e+00-6.1716497e-01j  1.78939140e+00-1.9770777e-02j]
 ...
 [-2.29853704e-05-1.4854236e-05j  9.01706244e-06+1.3912247e-05j
   9.99756503e-06+1.0044856e-05j ...  6.09892653e-03+2.1357394e-

In [9]:
def process_image(image):
    #make the images into spectrograms
    processed_img = tf.pow(tf.abs(image),2)
    #normalize the images?
    return processed_img

def process_path_A(path):
    #for training A, the label is just itself so we need to take that into account
    img = tf.io.read_file(path)
    img = tf.io.decode_raw(img,tf.complex64)
    img = tf.slice(img,begin=[1024//64],size=[IMAGE_SIZE]) # begin <- 1024//complex64
    img = tf.reshape(img,IMAGE_DIM)
    img = process_image(img)

    label = img
    return img, label

def process_path_B(path): #map_func
    #extract the 2d array and the label
    img = tf.io.read_file(path)
    img = tf.io.decode_raw(img,tf.complex64)
    img = tf.slice(img,begin=[1024//64],size=[IMAGE_SIZE]) # begin <- 1024//complex64
    img = tf.reshape(img,IMAGE_DIM)
    img = process_image(img)

    label = tf.strings.split( tf.strings.split( path, os.sep )[-1],sep="_" )[1]
    return img, label

In [10]:
dataset_A = dataset_for_training_A.map(process_path_A,num_parallel_calls=autotune)
dataset_B_train = dataset_for_training_B_train.map(process_path_B,num_parallel_calls=autotune)
dataset_B_test = dataset_for_training_B_test.map(process_path_B,num_parallel_calls=autotune)

In [11]:
next(iter(dataset_A))[1]

<tf.Tensor: shape=(513, 345, 1), dtype=float32, numpy=
array([[[1.6570969e+01],
        [1.0323366e+01],
        [4.1558967e+00],
        ...,
        [2.9841438e-01],
        [5.4224229e-01],
        [3.8677505e-01]],

       [[4.3974679e-02],
        [1.3621521e-01],
        [6.9864291e-01],
        ...,
        [2.1945457e-01],
        [3.9285794e-02],
        [8.5890628e-02]],

       [[1.9103150e-01],
        [1.2896417e-01],
        [1.6353089e-01],
        ...,
        [2.0201962e+00],
        [3.1482032e-01],
        [4.9994022e-01]],

       ...,

       [[4.0755740e-06],
        [2.0549001e-06],
        [9.3162220e-07],
        ...,
        [1.2263738e-01],
        [2.2376251e-01],
        [1.5041600e-01]],

       [[2.6261935e-02],
        [4.0671583e-02],
        [3.7511032e-02],
        ...,
        [8.3220236e-02],
        [3.9504219e-02],
        [1.0595024e-02]],

       [[2.2422839e-02],
        [5.6521870e-02],
        [5.4926306e-02],
        ...,
        [5.6126269e

In [12]:
def configure_for_performance(ds):
    ds = ds.cache()
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=autotune)
    return ds

dataset_A = configure_for_performance(dataset_A)
dataset_B_test = configure_for_performance(dataset_B_test)
dataset_B_train = configure_for_performance(dataset_B_train)

In [13]:

a = layers.Conv2D(16, (3,3), activation='relu', padding='valid', strides=2)
b = layers.Conv2D(8, (3,3), activation='relu', padding='same', strides=2)
c = layers.Conv2DTranspose(8, kernel_size=3, strides=2, activation='relu', padding='valid')
d = layers.Conv2DTranspose(16, kernel_size=3, strides=2, activation='relu', padding='valid')
e = layers.Conv2D(1, kernel_size=(3,3), activation='sigmoid', padding='valid')

input_shape = (1,513,345,1)
x = tf.random.normal(input_shape)

print(input_shape)
y = a(x)
print(y.shape)
y = b(y)
print(y.shape)
y = c(y)
print(y.shape)
y = d(y)
print(y.shape)
y = e(y)
print(y.shape)
y = e(d(c(b(a(x)))))
print(y.shape)

(1, 513, 345, 1)
(1, 256, 172, 16)
(1, 128, 86, 8)
(1, 257, 173, 8)
(1, 515, 347, 16)
(1, 513, 345, 1)
(1, 513, 345, 1)


## 2. Model initialization

In [14]:
class Autoencoder(tf.keras.Model):
#when the input shape is an odd number, even number of strides coupled with 'same' padding makes the output shape different from the imput shape.
#Therefore by trial and error this valid, same, valid, valid, valid padding structure was found, not sure if it is accurate
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = tf.keras.Sequential([
            layers.InputLayer(input_shape = IMAGE_DIM , batch_size= batch_size),
            layers.Conv2D(16, (3,3), activation='relu', padding='valid', strides=2),
            layers.Conv2D(8, (3,3), activation='relu', padding='same', strides=2), 
        ])
        self.decoder = tf.keras.Sequential([
            layers.Conv2DTranspose(8, kernel_size=3, strides=2, activation='relu', padding='valid'),
            layers.Conv2DTranspose(16, kernel_size=3, strides=2, activation='relu', padding='valid'),
            layers.Conv2D(1, kernel_size=(3,3), activation='sigmoid', padding='valid') 
        ])
    
    def __call__(self, x, training=False):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded
    
Model_A = Autoencoder()

In [15]:
class Classifier(tf.keras.Model):

    def __init__(self, Model_A):
        super(Classifier, self).__init__()
        #comes after the encoding
        self.classifer_layer = tf.keras.Sequential([
            layers.InputLayer(input_shape=[129,87,8], batch_size= batch_size),
            layers.Conv2D(8, (3,3), activation='relu', padding='same', strides=2),
            layers.Conv2D(4, (3,3), activation='relu', padding='same', strides=2),
            layers.Conv2D(2, (3,3), activation='relu', padding='same', strides=2),
            layers.Conv2D(1, (3,3), activation='relu', padding='same', strides=2),
            layers.Flatten(),
            layers.Dense(16, activation='relu'),
            layers.Dense(1, activation='softmax')
        ])
        self.Model_A = Model_A.encoder # Probablility of deepcopy, not sure. If accuracy is terrible this may be the suspect

    def __call__(self,x):
        compressed = self.Model_A(x)
        return self.classifer_layer(compressed)

Model_B = Classifier(Model_A)

In [16]:
Model_A.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss=tf.keras.losses.MeanSquaredError())

In [17]:
Model_B.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [18]:
# Create a callback function that saves the weights periodically, and shows the reconstructed images periodically
class Monitor_A(keras.callbacks.Callback):
    
    def on_epoch_end(self, epoch, logs=None):
        _, ax = plt.subplots(1,2, figsize=(21,7))
        for k,k in dataset_A.take(1):
            changed = self.model(k)
            clear_output(wait=True)
            ax[0, 0].imshow(k)
            ax[0, 1].imshow(changed)
            ax[0, 0].set_title("Input image")
            ax[0, 1].set_title("Translated image")
            ax[0, 0].axis("off")
            ax[0, 1].axis("off")
        
        if (epoch) % 5 == 0:
            self.model.save_weights(pathlib.Path(save_weights_A_path/"{epoch:04}.h5"))
    
        plt.show()
        plt.close()

In [19]:
# Create a callback function that saves the weights periodically, and shows the reconstructed images periodically
class Monitor_B(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        
        if (epoch) % 5 == 0:
            self.model.save_weights(pathlib.Path(save_weights_B_path/"{epoch:04}.h5"))


## 3. Training & Evaluation

In [ ]:
with tf.device('/gpu:0'):
    history = Model_A.fit(dataset_A, batch_size=batch_size, epochs=100, callbacks=[Monitor_A()])

In [ ]:
Model_A.fit(dataset_A, batch_size=batch_size, epochs=100, callbacks=[Monitor_A()])

In [ ]:
plt.plot(history.history["loss"], label="Training Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.legend()

In [ ]:
with tf.device('/gpu:0'):
    history = Model_B.fit(dataset_B_train, batch_size=batch_size, epochs=100, callbacks=[Monitor_A], validation_data=dataset_B_test)

![](/Experiment1abstract.jpg?raw=true)